## Cleaning scraped game data

Here we show how to use the data obtained with Scrapy. In order to use it for data analysis and game outcome predictions, we first need to clean the data. 

Let's start with importing the packages we'll use:

In [1]:
import pandas as pd # Dataframes
import numpy as np # number crunching, matrices and all that

Let's now import the scraped data and perform a first simple cleaning step:
1. We import the `.csv` file to a pandas data frame.
2. There are games that were scraped multiple times because multiple of the selected top players were involved in them (a game might pop up in our data up to 10 times because of this). As these duplicates would skew the statistics, we remove them via `drop_duplicates`, using the starting time (`timestamp`) and the duration (`duration`) as unique identifiers.
3. We reset the index of the data frame, which can be done explicitly (see commented line) or implicitly when removing duplicates via `ignore_index=True`.

In [5]:
df = pd.read_csv('../Scrapy/loltest/games.csv')
df.drop_duplicates(subset=['duration', 'timestamp', 'team_1', 'team_2', 'winner'], inplace=True, keep='first', ignore_index=True)
# df.reset_index(drop=True, inplace=True)
print(len(df))
df.head()

19709


,duration,server,summoner_name,team_1,team_2,timestamp,winner
0,26m 18s,na,AnDa,"Camille,Gragas,Yone,Jhin,Brand","Shen,Graves,Sylas,Kai'Sa,Karma",2020-12-23 14:12:18,Team 2
1,20m 42s,na,AnDa,"Camille,Nidalee,Galio,Jhin,Anivia","Riven,Hecarim,Xerath,Miss Fortune,Vel'Koz",2020-12-23 13:19:48,Team 1
2,15m 20s,na,AnDa,"Irelia,Ekko,Annie,Kai'Sa,Thresh","Aatrox,Jarvan IV,Orianna,Aphelios,Maokai",2020-12-22 15:11:07,Team 1
3,30m 34s,na,AnDa,"Graves,Kindred,Sylas,Vayne,Bard","Renekton,Nidalee,Ekko,Sivir,Pyke",2020-12-22 14:46:00,Team 2
4,29m 58s,na,AnDa,"Camille,Lee Sin,Lucian,Kai'Sa,Leona","Jhin,Nidalee,Yone,Orianna,Pantheon",2020-12-22 13:59:22,Team 2


As we can see in the print-out of the data frame head above, we now have unique games in `df`, with the columns `duration`, `server`, `summoner_name`, `team_1`, `team_2`, `timestamp` and `winner`. We will usually discard the server, player (summoner) and time information in our analysis.

In order to capture the roles of the played champions, which currently is implicitly stored in their order in `team_1` and `team_2`, we create 10 new columns - 5 for the red and blue team each - and store the champions individually:

In [6]:
# These are the roles, in the same order as they are stored in team_1 and team_2.
roles = ['Top', 'Jng', 'Mid', 'Adc', 'Sup']
# For both teams...
for team_color, team_attr in zip(['B', 'R'], ['team_1', 'team_2']):
    # ...decompose the column in a data frame of champion names...
    team = df[team_attr].str.split(',', expand=True)
    # ...and for all 5 roles, store the role column in the corresponding column of df.
    for i, role in enumerate(roles):
        df[f"{team_color}{role}"] = team[i]
df.drop(columns=['team_1', 'team_2'], inplace=True)

_Note on performance_: The above splitting of `team_1` and `team_2` is done for the entire data frame "at once" as we are using an internal pandas function (`pd.Series.str.split`) and then assign the full columns to the new role columns `BTop`, `BJng`... of `df`.

Let's now rewrite the `winner` column to use `'Blue'` and `'Red'` instead of `'Team 1'` and `'Team 2'`, and drop the above mentioned columns of information we do not take into account.

We also already can do a first step of data analysis and consider some stats:

In [7]:
df['winner'] = df.apply(lambda x: 'Blue' if x.winner=='Team 1' else 'Red', axis=1)
df.drop(['server', 'summoner_name', 'duration', 'timestamp'],axis=1,inplace=True)
# Some statistics:
num_games = len(df)   # Total number of games
num_blue_wins = len(df[df['winner']=='Blue'])     # No of games blue won
num_red_wins = len(df[df['winner']=='Red'])      # No of games red won
assert num_red_wins + num_blue_wins == num_games # Make sure we do not have a bad row without winner or such.
blue_winrate = num_blue_wins/num_games
red_winrate = num_red_wins/num_games
print(f"There are {num_games} games recorded, the blue team won {num_blue_wins},",
      f"the red team won {num_red_wins} of these games.",
      f"\nThis yields win rates of {blue_winrate*100:.2f}% (blue) and {red_winrate*100:.2f}% (red).")

There are 19709 games recorded, the blue team won 9936, the red team won 9773 of these games. 
This yields win rates of 50.41% (blue) and 49.59% (red).


### Looking at the champion stats
Now we will prepare a second important data frame using the data above: The statistics per champion.
To get the unique champion names, let's use `np.unique` on all role columns in `df`.

In [8]:
Blue = [f'B{role}' for role in roles]
Red = [f'R{role}' for role in roles]
champions = np.unique(df[Blue+Red])
# cd = pd.DataFrame(champions, columns=['Champion'])

Now we compute the statistics per champion.

In order to speed up the process by using `dict` lookups (which are very fast), we will not do the following steps in the `cd` data frame directly but make use of four separate dictionaries that capture the numbers of games/wins on the blue/red side for each champion. We also memorize the roles that the champions were played in, using a `dict` with `dict`s as values.

To actually count the values we are interested in, we iterate over the data frame of games `df` _once_. For each row, we iterate over the roles and add to the counters in `blue_played` and `red_played` for the champions played on the respective side. We also memorize the role that each champion was played in. In order to count the wins on either side, we make use of python's automatic type casting and add the boolean `winner_is_blue`/`winner_is_red` to the counters in `blue_won` and `red_won`.

In [9]:
blue_played = {champ: 0 for champ in champions}
blue_won = {champ: 0 for champ in champions}
red_played = {champ: 0 for champ in champions}
red_won = {champ: 0 for champ in champions}
roles_played = {champ: {role: 0 for role in roles} for champ in champions}

for _, row in df.iterrows():
    winner_is_blue = row.winner=='Blue'
    winner_is_red = not winner_is_blue
    for blue_role in Blue:
        champ = row[blue_role]
        blue_played[champ] += 1
        blue_won[champ] += winner_is_blue
        # Strip the "B"/"R" from blue_role to get the role
        roles_played[champ][blue_role[1:]] += 1
    for red_role in Red:
        champ = row[red_role]
        red_played[champ] += 1
        red_won[champ] += winner_is_red
        roles_played[champ][red_role[1:]] += 1

Before storing everything in a data frame, let's figure out which were the most played roles per champion. For this, we iterate over the champions and sort the roles by their occurences for each champion. The `number_of_roles_to_record` most played roles and their counters are then stored in individual lists and linked to keys, for example `"Role1"` and `"#Role1"`, in a dictionary:

In [10]:
number_of_roles_to_record = 2 # We use 2 roles, could use up to all 5
ordered_roles_played = [[] for _ in range(number_of_roles_to_record)]
numbers_roles_played = [[] for _ in range(number_of_roles_to_record)]
for i, champ in enumerate(champions):
    # This is a list of tuples (role, #plays in the role):
    roles_for_this_champ = list(roles_played[champ].items()) 
    # sort by number of plays, in descending order (reverse=True)
    sorted_roles_for_this_champ = sorted(roles_for_this_champ, key=lambda x: x[1], reverse=True)
    
    # Now let's record the sorted tuples as order of most played roles (and their # of plays) 
    for j in range(number_of_roles_to_record):
        ordered_roles_played[j].append(sorted_roles_for_this_champ[j][0]) # Record the role
        numbers_roles_played[j].append(sorted_roles_for_this_champ[j][1]) # Record the # of plays
        
most_played_roles = {f"Role{j+1}": ordered_roles_played[j] for j in range(number_of_roles_to_record)}
most_played_numbers = {f"#Role{j+1}": numbers_roles_played[j] for j in range(number_of_roles_to_record)}

Having all statistics sorted out, we can wrap everything up in a data frame. Because of the way we stored the most played roles above, we have a flexible pipeline that will generate the data frame for any number of most-played roles we want to store per champion.

In [17]:
cd = pd.DataFrame({
    'Champion': champions,
    'BluePlayed': [blue_played[champ] for champ in champions],
    'BlueWon': [blue_won[champ] for champ in champions],
    'RedPlayed': [red_played[champ] for champ in champions],
    'RedWon': [red_won[champ] for champ in champions],
    **most_played_roles,
    **most_played_numbers,
})

We conclude the first round of data analysis by computing the total number of games played and the win rate on either side as well as in total, for each champion. For this, the column-wise operations on a data frame are very handy:

In [18]:
cd['TotalPlayed'] = cd['BluePlayed'] + cd['RedPlayed']
cd['Bluewinrate'] = cd['BlueWon'] / cd['BluePlayed']
cd['Redwinrate'] = cd['RedWon'] / cd['RedPlayed']
cd['Totalwinrate'] = (cd['BlueWon'] + cd['RedWon']) / cd['TotalPlayed']

The resulting data frame looks like this:

In [19]:
cd

,Champion,BluePlayed,BlueWon,RedPlayed,RedWon,Role1,Role2,#Role1,#Role2,TotalPlayed,Bluewinrate,Redwinrate,Totalwinrate
0,Aatrox,1134,615,1182,579,Top,Mid,2001,191,2316,0.542328,0.489848,0.515544
1,Ahri,247,119,246,119,Mid,Sup,459,19,493,0.481781,0.483740,0.482759
2,Akali,2025,1019,1976,908,Mid,Top,2529,1443,4001,0.503210,0.459514,0.481630
3,Alistar,1469,784,1524,776,Sup,Mid,2970,14,2993,0.533696,0.509186,0.521216
4,Amumu,190,92,181,93,Jng,Sup,356,7,371,0.484211,0.513812,0.498652
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,Zed,537,255,506,255,Mid,Top,910,82,1043,0.474860,0.503953,0.488974
149,Ziggs,100,51,100,54,Adc,Mid,95,91,200,0.510000,0.540000,0.525000
150,Zilean,230,110,204,91,Sup,Mid,364,53,434,0.478261,0.446078,0.463134
151,Zoe,789,424,774,386,Mid,Adc,1507,27,1563,0.537389,0.498708,0.518234


For other parts of the project we will want to come back to this data. Let's store it in a new `.csv` file.

In [50]:
cd.to_csv('../Data/ChampionStatsDemo.csv',index=False)